In [2]:
import cv2
import numpy as np
from keras.models import load_model
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image


In [3]:
model = load_model(r"C:\Users\soodv\OneDrive\Desktop\Crop disease prediction Project\DenseNet121.h5")

In [4]:
# Set the confidence threshold.
confidence_threshold = 0.9

# Start the webcam.
cap = cv2.VideoCapture(0)

In [5]:
class_names = ['Apple___Apple_scab',
               'Apple___Black_rot',
               'Apple___Cedar_apple_rust',
               'Apple___healthy',
               'Blueberry___healthy',
               'Cherry_(including_sour)___Powdery_mildew',
               'Cherry_(including_sour)___healthy',
               'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
               'Corn_(maize)__Common_rust',
               'Corn_(maize)___Northern_Leaf_Blight',
               'Corn_(maize)___healthy',
               'Grape___Black_rot',
               'Grape__Esca(Black_Measles)',
               'Grape__Leaf_blight(Isariopsis_Leaf_Spot)',
               'Grape___healthy',
               'Orange__Haunglongbing(Citrus_greening)',
               'Peach___Bacterial_spot',
               'Peach___healthy',
               'Pepper,bell__Bacterial_spot',
               'Pepper,bell__healthy',
               'Potato___Early_blight',
               'Potato___Late_blight',
               'Potato___healthy',
               'Raspberry___healthy',
               'Soybean___healthy',
               'Squash___Powdery_mildew',
               'Strawberry___Leaf_scorch',
               'Strawberry___healthy',
               'Tomato___Bacterial_spot',
               'Tomato___Early_blight',
               'Tomato___Late_blight',
               'Tomato___Leaf_Mold',
               'Tomato___Septoria_leaf_spot',
               'Tomato___Spider_mites Two-spotted_spider_mite',
               'Tomato___Target_Spot',
               'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
               'Tomato___Tomato_mosaic_virus',
               'Tomato___healthy']

In [6]:
import cv2
import numpy as np
from keras.models import load_model
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image

class FrameConfig:
    def __init__(self, width=224, height=224, min_size=32, max_size=1024):
        self.min_size = min_size
        self.max_size = max_size
        self._width = None
        self._height = None
        # Set initial values through properties to ensure validation
        self.width = width
        self.height = height
    
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, value):
        if not isinstance(value, int):
            raise ValueError("Width must be an integer")
        if not self.min_size <= value <= self.max_size:
            raise ValueError(f"Width must be between {self.min_size} and {self.max_size}")
        self._width = value
    
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, value):
        if not isinstance(value, int):
            raise ValueError("Height must be an integer")
        if not self.min_size <= value <= self.max_size:
            raise ValueError(f"Height must be between {self.min_size} and {self.max_size}")
        self._height = value
    
    @property
    def dimensions(self):
        return (self.width, self.height)

def initialize_model(model_path):
    try:
        return load_model(model_path)
    except Exception as e:
        raise RuntimeError(f"Failed to load model from {model_path}: {str(e)}")

def process_frame(frame, frame_config, model, class_names, confidence_threshold=0.9):

    try:
        # Resize frame according to config
        resized_frame = cv2.resize(frame, frame_config.dimensions)
        
        # Preprocess the frame
        processed = image.img_to_array(resized_frame)
        processed = np.expand_dims(processed, axis=0)
        processed = preprocess_input(processed)
        
        # Make prediction
        predictions = model.predict(processed)
        predicted_class = np.argmax(predictions)
        confidence = predictions[0][predicted_class]
        
        # Add detection results to frame
        if confidence > confidence_threshold:
            label = class_names[predicted_class]
            text = f"{label} ({confidence:.2f})"
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
        else:
            cv2.putText(frame, "Not Identified", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            
    except Exception as e:
        print(f"Error processing frame: {str(e)}")
        cv2.putText(frame, "Error Processing Frame", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
    
    return frame

def main():
    # Initialize configuration
    try:
        frame_config = FrameConfig(width=224, height=224)
    except ValueError as e:
        print(f"Invalid frame configuration: {str(e)}")
        return

    # Load model
    model_path = r"D:\coding\Acwoc Contributions\DenseNet-121-Model-for-Crop-Disease-Detection\DenseNet121.h5"
    try:
        model = initialize_model(model_path)
    except RuntimeError as e:
        print(f"Model initialization failed: {str(e)}")
        return

    # Initialize camera
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Failed to open camera")
        return

    print(f"Starting detection with frame size: {frame_config.dimensions}")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            break

        # Process frame
        processed_frame = process_frame(frame, frame_config, model, class_names)
        
        # Display results
        cv2.imshow('Real Time Crop Disease Detection', processed_frame)
        
        # Check for quit command
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

1/1 [==============================] - 0s 145ms/step


In [7]:
# Release the webcam and close all windows.
cap.release()
cv2.destroyAllWindows()